# Lập trình thống kê với Python

# Bài 5: Kiểm định Chi bình phương

### BS. Lê Ngọc Khả Nhi

# Giới thiệu

Trong bài này, Nhi tạo ra 1 công cụ phân tích bảng chéo, bao gồm kiểm định $\chi^2$.

Xin nói rõ, công cụ mà ta sắp tạo ra không dựa trên kiểm định $\chi^2$ của Pearson mà các bác sĩ đã học trong trường lớp, nhưng là một phương pháp phổ quát hơn là Power divergence statistic và goodness of fit test, do 2 tác giả Noel Cressie và Timothy R. C. Read lập ra vào năm 1984.

Trong Power divergence của Cressie và Read có một tham số $\lambda$, gồm các giá trị có thể:

+ $\lambda$ = 1 tương đương với kiểm định Chi2 cổ điển cùa Pearson

+ $\lambda$ = 0 tương đương với G-test (log-likelihood ratio test)

+ $\lambda$ = -0.5 tương đương với trị số thống kê của Freeman và Tukey

+ $\lambda$ = -1 tương đương với kiểm định log-likelihood ratio hiệu chỉnh

+ $\lambda$ = -2 tương đương với trị số thống kê của Neyman

+ $\lambda$ = 2/3 là giá trị đề xuất bởi Cressie và Read năm 1984

Lưu ý: Tuy "kiểm định $\chi^2$ gằn liền với bảng chéo (contingency table), bản chất của nó là kiểm tra về tính hợp lý dữ liệu của mô hình (goodness of fit), bảng chéo là một trường hợp đặc biệt của mô hình. Theo định nghĩa, phân phối $\chi^2$ có bản chất là tổng bình phương của những sai số residual giữa mô hình và quan sát thực tế). Phân phối $\chi^2$ được dùng rất phổ biến trong kiểm định thống kê; có vài chục loại kiểm định mà trị số thống kê được mô tả bằng phân phối này.

Về bản chất, giả thuyết H0 và H1 của một kiểm định $\chi^2$ liên quan đến tính độc lập giữa 2 biến định tính, hoặc tính phù hợp của mô hình. Việc so sánh tỉ lệ hay tần suất giữa các phân nhóm chỉ là câu hỏi/suy luận chủ quan của người dùng, ta lợi dụng kiểm định $\chi^2$  như một phương tiện.

# Bài toán minh họa

Bộ số liệu này có nguồn gốc từ 1 nghiên cứu thử nghiệm lâm sàng của Edwards Koch năm 1988, với mục tiêu khảo sát hiệu quả điều trị bệnh Viêm khớp của một loại thuốc (nhóm Treatment), so với nhóm sử dụng giả dược (placebo). Hiệu quả điều trị được đánh giá như một biến định tính với 3 bậc giá trị : Cải thiện rõ rệt (marked), cải thiện tương đối (Some) và không có cải thiện (None) triệu chứng của bệnh.

Giả thuyết cần chứng minh có thể phát biểu theo 2 cách:

1) Có sự khác biệt về tỉ lệ/tần suất bệnh nhân có cải thiện triệu chứng giữa 2 phân nhóm : Dùng thuốc và dùng Giả dược. Cách tư duy này sẽ dẫn chúng ta đến bài toán so sánh tỉ lệ điều trị thành công giữa 2 phân nhóm (Treatment vs Placebo), ghi chú là tỉ lệ này chính là tỉ số giữa số bệnh nhân có cải thiện chia cho tổng số bệnh nhân ở từng phân nhóm điều trị.

Hoặc 

2) Có mối liên hệ giữa yếu tố phân nhóm điều trị và mức độ cải thiện triệu chứng; Ghi chú: liên hệ ở đây được hiểu ngầm – là về tần suất. 

In [191]:
warning_status = "ignore"
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)

import numpy as np
import pandas as pd

import plotly.express as px

from patsy import dmatrices  # Tạo design matrix

from scipy.stats.contingency import expected_freq
from scipy.stats import power_divergence, binom, chi2 as sp_chi2, chi2_contingency

In [194]:
data = pd.read_csv("https://raw.github.com/vincentarelbundock/Rdatasets/master/csv/vcd/Arthritis.csv", index_col=0)

data

,ID,Treatment,Sex,Age,Improved
1,57,Treated,Male,27,Some
2,46,Treated,Male,29,None
3,77,Treated,Male,30,None
4,17,Treated,Male,32,Marked
5,36,Treated,Male,46,Marked
...,...,...,...,...,...
80,32,Placebo,Female,66,None
81,42,Placebo,Female,66,None
82,15,Placebo,Female,66,Some
83,71,Placebo,Female,68,Some


# Class Contingency_test

Nhi dùng Python OOP để tạo ra 1 class Contingency_test(), nhận dữ liệu đầu vào là pandas dataframe; và tên 2 biến định tính x,y mà ta muốn phân tích về liên hệ giữa chúng.

Đầu tiên ta kiểm tra x, y có nằm trong dataframe hay không; nếu có, ta tạo ra 2 bảng chéo chứa tần suất quan sát và giả định của tổ hợp giữa các bậc giá trị của x, y.

Bảng chéo này không bắt buộc là 2x2, nhưng có kích thước tùy theo bậc giá trị của x, y;

Method tree_plot cho phép vẽ một biểu đồ mosaic plot (hay tree plot) tương tác, sử dụng plotly.

Method Chi_squared_test thực hiện 6 phiên bản kiểm định $\chi^2$ khác nhau dựa vào Power divergence của Cressie và Read (6 giá trị lambda khác nhau), mỗi kiểm định như vậy xuất ra giá trị p_value, effect-size là Cramer's V và kết luận có thể phủ nhận H0 hay không.

Người dùng có thể tùy chọn có áp dụng phương pháp hiệu chỉnh Yates hay không.

In [192]:
class Contingency_test():
    
    def __init__(self, data, x, y):
        
        x = x.strip()
        y = y.strip()
        self.data = data
        
        from scipy.stats.contingency import expected_freq
        
        try:
            assert all(i in data.columns for i in (x, y))
        except AssertionError:
            print('Lỗi: ít nhất 1 biến không tồn tại trong dataframe')
        else:
            self.observed = pd.crosstab(data[x], data[y])

            self.expected = pd.DataFrame(expected_freq(self.observed), 
                                         index=self.observed.index,
                                         columns=self.observed.columns)

            for xtab, name in zip([self.observed, self.expected], 
                                          ['Quan sát', 'Giả định']):
                if (xtab < 5).any(axis=None):
                    print(f'Lưu ý: Xuất hiện giá trị < 5 trong bảng {name} !')
            
            print('Bảng chéo')
            print('-'*10)
            print(self.observed)
            
    def tree_plot(self):
        x = self.observed.index.name
        y = self.observed.columns.name
        y_lev = self.observed.columns
        df = self.observed.copy()
        df[x] = df.index
        
        df_long = pd.melt(df, id_vars=x, value_vars=y_lev)
        
        fig = px.treemap(df_long, path=[x,y], 
                         values='value',
                         color='value',
                         color_continuous_scale='Blues')
        fig.show()
        
    def Chi_squared_test(self, yate_adj = True):
        
        dof = float(self.expected.size - sum(self.expected.shape) + \
                    self.expected.ndim - 1)
        
        n = self.observed.values.sum()
        
        if dof == 1 and yate_adj:
            # Hiệu chỉnh Yates
            self.observed = self.observed + 0.5 * np.sign(self.expected - self.observed)
            
        ddof = self.observed.size - 1 - dof
        
        methods = ["Pearson", "Cressie-Read",  
                   "log-likelihood", "Freeman-Tukey",
                   "mod-log-likelihood", "Neyman"]
        
        stats = []
        
        for met, lambda_ in zip(methods, [1.0, 2/3, 0.0, -1 / 2, -1.0, -2.0]):
            if dof == 0:
                chi2, p, cramer, power = 0.0, 1.0, np.nan, np.nan
            else:
                chi2, p = power_divergence(self.observed, 
                                           self.expected, 
                                           ddof=ddof,
                                           axis=None, 
                                           lambda_=lambda_)
                
                cramer = np.sqrt(chi2 / (n * (min(self.expected.shape) - 1)))
                
            stats.append({'Method': met, 
                  u'\u03BB': lambda_,
                  'df': dof,
                  u'\u03C7'+u'\u00b2': chi2,
                  "Cramer's V": cramer,
                  'p': p,
                  'H0 reject': True if p < 0.05 else False})
        
        out = pd.DataFrame(stats)[['Method', 
                             u'\u03BB',
                             'df', 
                             u'\u03C7'+u'\u00b2', 
                             "Cramer's V",
                             'p',
                             'H0 reject',]]
        
        return out.style.hide_index()        

# Áp dụng 

## Trường hợp 1: Kiểm tra liên hệ giữa kết quả và điều trị

In [195]:
xtab_1 = Contingency_test(data, 'Treatment','Improved')

Bảng chéo
----------
Improved   Marked  None  Some
Treatment                    
Placebo         7    29     7
Treated        21    13     7


In [196]:
xtab_1.tree_plot()

In [197]:
xtab_1.Chi_squared_test()

Method,λ,df,χ²,Cramer's V,p,H0 reject
Pearson,1.000000,2.000000,13.055020,0.394230,0.001463,True
Cressie-Read,0.666667,2.000000,13.153104,0.395708,0.001393,True
log-likelihood,0.000000,2.000000,13.529807,0.401334,0.001154,True
Freeman-Tukey,-0.500000,2.000000,13.987643,0.408068,0.000918,True
mod-log-likelihood,-1.000000,2.000000,14.618370,0.417167,0.000669,True
Neyman,-2.000000,2.000000,16.517901,0.443443,0.000259,True


## Trường hợp 2: Kiểm tra liên hệ giữa Giới tính và Hiệu quả điều trị

In [198]:
xtab_2 = Contingency_test(data, 'Sex','Improved')

Lưu ý: Xuất hiện giá trị < 5 trong bảng Quan sát !
Lưu ý: Xuất hiện giá trị < 5 trong bảng Giả định !
Bảng chéo
----------
Improved  Marked  None  Some
Sex                         
Female        22    25    12
Male           6    17     2


In [199]:
xtab_2.tree_plot()

In [200]:
xtab_2.Chi_squared_test()

Method,λ,df,χ²,Cramer's V,p,H0 reject
Pearson,1.000000,2.000000,4.840678,0.240056,0.088891,False
Cressie-Read,0.666667,2.000000,4.883236,0.241109,0.087020,False
log-likelihood,0.000000,2.000000,5.013082,0.244294,0.081550,False
Freeman-Tukey,-0.500000,2.000000,5.155427,0.247738,0.075947,False
mod-log-likelihood,-1.000000,2.000000,5.343568,0.252218,0.069129,False
Neyman,-2.000000,2.000000,5.894485,0.264901,0.052484,False


# Kết luận:

Chúng ta vừa tạo ra 1 công cụ phân tích bảng chéo, dựa vào lý thuyết về Power divergence của Read (1984), tương ứng với 6 biến thể của kiểm định $\chi^2$ bao gồm phương pháp cổ xưa nhất của Pearson.

### Tài liệu tham khảo:

Freeman, M.F. and J.W. Tukey (1950), Transformations related to the angular and the square root, Ann. Math. Statist. 21, 607-611.

Campbell B. Read. Freeman--Tukey chi-squared goodness-of-fit statistics. Statistics & Probability Letters, 1993, vol. 18, issue 4, 271-278.

J. Neyman and E. S. Pearson, On the use and interpretation of certain test criteria for purposes of statistical inference: Part i, Biometrika 20A (1928) 175–240.

Xiangpan Ji, Wenqiang Gu, Xin Qian, Hanyu Wei, Chao Zhang. Combined Neyman-Pearson Chi-square: An Improved Approximation to the Poisson-likelihood Chi-square. Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment 2020; 961:163677

Noel Cressie and Timothy R. C. Read. Multinomial goodness‐of‐fit tests. Journal of the Royal Statistical Society: Series B (Methodological), 1984; 46(3), 440-464.

Frank Yates. Contingency Tables Involving Small Numbers and the $\chi^2 test$. Supplement to the Journal of the Royal Statistical Society, 1934; 1: 217-235.